<a href="https://colab.research.google.com/github/RiyadAhmedCS/CSNEA2022/blob/main/model_128x128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###times
######w/ xla + mixed precision : 3m10s
######w/ xla : 2m40s
######w/o xla + mixed precision: 2m41s
######w tpu + xla:

Code

In [ ]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
import os
os.environ["TF_CCP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.optimizer.set_jit(True)

In [ ]:
dataset = keras.preprocessing.image_dataset_from_directory(directory ="/content/drive/MyDrive/nea_files/newimgs128",
                                                           label_mode=None,
                                                           image_size=(128, 128),
                                                           batch_size=32,
                                                           shuffle=True
                                                           
                                                           ).map(lambda x: x/255)

Found 250 files belonging to 1 classes.


In [ ]:
if os.path.isdir('/content/drive/MyDrive/nea_files/newimgs128/model_Discriminator'):
  discriminator = keras.models.load_model('/content/drive/MyDrive/nea_files/newimgs128/model_Discriminator.h5')
  print("model loaded")
else:
  print("failed")
  discriminator = keras.Sequential(
      [
          keras.Input(shape=(128, 128, 3)), 
          layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(0.2),
          layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(0.2),
          layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(0.2),
          layers.Conv2D(512, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(0.2),
          layers.Flatten(),
          layers.Dropout(0.2),
          layers.Dense(1, activation="sigmoid")
          #Sigmoid use to determine if image real or fake
      ]
  )

print(discriminator.summary())

model loaded
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        3136      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 128)       131200    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 256)       524544    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 16, 16, 256)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 5

In [ ]:
if os.path.isdir('/content/drive/MyDrive/nea_files/newimgs128/model_Generator'):
  generator = keras.models.load_model('/content/drive/MyDrive/nea_files/newimgs128/model_Generator')
  print("model loaded")

else:
  latent_dim = 128

  generator = keras.Sequential(
      [
          layers.Input(shape=(latent_dim,)),
          layers.Dense(8*8*128),
          layers.Reshape((8,8,128)),
          layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(0.2),
          layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(0.2),
          layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(0.2),
          layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(0.2),
          layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
      ]
  )
print(generator.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 8192)              1056768   
                                                                 
 reshape (Reshape)           (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      262272    
 nspose)                                                         
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 256)      524544    
 ranspose)                                                       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32, 32, 256)      

In [ ]:
opt_gen = keras.optimizers.Adam(1e-4)
opt_disc = keras.optimizers.Adam(1e-4)
loss_fn = keras.losses.BinaryCrossentropy()

for epoch in range(1,2):
  generator.save("/content/drive/MyDrive/nea_files/newimgs128/model_Generator.h5")
  generator.compile()
  discriminator.save("/content/drive/MyDrive/nea_files/newimgs128/model_Discriminator.h5")
  discriminator.compile()
  for i, real in enumerate(tqdm(dataset)):
    batch_size = real.shape[0]
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    fake = generator(random_latent_vectors)

    if i % 100 == 0:
      img = keras.preprocessing.image.array_to_img(fake[0])
      img.save(f"/content/drive/MyDrive/nea_files/generated_imgs/generated_img{epoch}_{i}.png")
    with tf.GradientTape() as disc_tape:
      loss_disc_real = loss_fn(tf.ones(batch_size, 1), discriminator(real))
      loss_disc_fake = loss_fn(tf.zeros(batch_size, 1), discriminator(fake))
      loss_disc = (loss_disc_real + loss_disc_fake) / 2

    grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
    opt_disc.apply_gradients(
        zip(grads, discriminator.trainable_weights)
        )
    
    with tf.GradientTape() as gen_tape:
      fake = generator(random_latent_vectors)
      output = discriminator(fake)
      loss_gen = loss_fn(tf.ones(batch_size, 1), output)

    grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
    opt_gen.apply_gradients(
        zip(grads, generator.trainable_weights)
        )
    


100%|██████████| 8/8 [01:01<00:00,  7.74s/it]


In [ ]:
img = keras.preprocessing.image.array_to_img(fake[0])
img.save(f"/content/drive/MyDrive/nea_files/generated_imgs/generated_img{epoch}_{i}.png")